# Vietnam Coastlines Combined


* Load stack of all available Landsat 5, 7, 8 and 9 satellite imagery for a location 
* Convert each satellite image into a remote sensing water index (MNDWI)
* For each satellite image, model ocean tides into a grid based on exact time of image acquisition
* Interpolate tide heights into spatial extent of image stack using the [FES2014 global tide model](https://github.com/GeoscienceAustralia/dea-coastlines/wiki/Setting-up-tidal-models-for-DEA-Coastlines)
* Mask out high and low tide pixels by removing all observations acquired outside of 50 percent of the observed tidal range centered over mean sea level
* Combine tidally-masked data into annual median composites representing the most representative position of the coastline at approximately mean sea level each year
* Apply morphological extraction algorithms to mask annual median composite rasters to a valid coastal region
* Extract waterline vectors using subpixel waterline extraction ([Bishop-Taylor et al. 2019b](https://doi.org/10.3390/rs11242984))
* Compute rates of coastal change at every 30 m using linear regression

This is an interactive version of the code intended for prototyping; to run this analysis at scale, use the [command line tools](DEACoastlines_generation_CLI.ipynb).

---

## Getting started

Set working directory to top level of repository to ensure links work correctly:

In [ ]:
cd ..

### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["USE_PYGEOS"] = "0"

# Load DEA Coastlines and DEA tools code
from coastlines.utils import get_study_site_geometry, load_config
from coastlines.combined import (
    load_and_mask_data_with_stac,
    export_results,
    filter_by_tides,
    generate_yearly_composites,
    mask_pixels_by_tide,
    sanitise_tile_id,
)
from coastlines.vector import (
    points_on_line,
    annual_movements,
    calculate_regressions,
    all_time_stats,
    contours_preprocess,
    contour_certainty,
    points_certainty,
)

# Load other libraries
from pathlib import Path
import geopandas as gpd
from datacube.utils.dask import start_local_dask
from dea_tools.spatial import subpixel_contours
from odc.stac import configure_s3_access

In [ ]:
# Hide warnings. Don't run this cell to see the warnings.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create local dask client for parallelisation
dask_client = start_local_dask(
    n_workers=4, threads_per_worker=8, mem_safety_margin="2GB"
)

# Configure S3 access including request payer
_ = configure_s3_access(requester_pays=True, cloud_defaults=True)

print(dask_client.dashboard_link.replace("/user", "https://hub.asia.easi-eo.solutions/user"))

## Setup


### Set analysis parameters

In [ ]:
# Study area selection
# study_area = "13,45" # North
study_area = "9,19"  # South west
# study_area = "18,32" # Central

# Issues!

# Still outstanding issues:
# study_area = "34,21"  # Very noisy and more of a reef than an island. No ESA WC data.
# study_area = "33,22"  # Works, but noisy and some of the area is being clipped off
# study_area = "32,18"  # Not enough data.

# study_area = "6,20"  # Noisy ocean, so look at different masking
# study_area = "33,21"  # Very noisy and nothing is generated.

# Atoll with lots of noise
# study_area = "28,15"
# study_area = "29,15"  # Failed again.

# Atolls...
# study_area = "31,19"
# study_area = "32,23"
# study_area = "33,24"
# study_area = "26,35"

# Important atolls
# study_area = "29,24"  # Alex is exploring this one
# study_area = "27,21"
# study_area = "24,19"
# study_area = "18,22"

# Config
version = "testing"
start_year = 2012
end_year = 2022
baseline_year = 2021
water_index = "mndwi"
index_threshold = 0.0

config_path = "configs/vietnam_coastlines_config_development.yaml"

# Tide data and config
home = Path("~")
tide_data_location = f"{home}/tide_models"
tide_centre = 0.0

# Output config
output_dir = Path(f"data/interim/vector/{version}/{sanitise_tile_id(study_area)}_{version}")
output_dir.mkdir(exist_ok=True, parents=True)
output_cache_zarr = output_dir / f"{sanitise_tile_id(study_area)}_{version}_combined_ds.zarr"

# Load analysis params from config file
config = load_config(config_path=config_path)

# Load the geometry from the grid used for the location
geometry = get_study_site_geometry(config["Input files"]["grid_path"], study_area)

# BBOX and other query parameters
bbox = list(geometry.buffer(0.05).bounds.values[0])

# Use the USGS STAC API to identify scenes to load
query = {
    "bbox": bbox,
    "datetime": (str(start_year - 1), str(end_year + 1)),
}


In [ ]:
# View the AoI
geometry.explore(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
)

## Loading data

### Create spatiotemporal query using a STAC API as the backend
This establishes the spatial and temporal extent used to search for Landsat satellite data.


In [ ]:
# Load the data, mask it and generate composites.
# This is lazy-loaded, so no data is actually loaded yet.
ds, items = load_and_mask_data_with_stac(config, query, include_nir=True, debug=True)

print(f"Found {len(items)} items")

ds

### Remove scenes that are completely at an extreme high or low tide

For each satellite timestep, model tide heights into a low-resolution 5 x 5 km grid (matching resolution of the FES2014 tidal model), then use that to filter out scenes with all high or low tides (those outside the middle 50% of tides)

In [ ]:
filtered = filter_by_tides(ds, tide_data_location, tide_centre)

print(f"Reduced from {len(ds.time)} to {len(filtered.time)} timesteps")

### Load data into memory

This step loads daily cloud masked data into memory

In [ ]:
# Optionally load the daily dataset into memory, either do this here or
# down below for the combined dataset. This takes more memory, but is
# faster. The below one results in a big, complex dask graph, but saves
# a fair bit of memory.
filtered = filtered.compute()
filtered

### Tidal masking

Here we do a per-pixel mask of the extreme tide pixels. This is done with data loaded into memory, to keep things efficient on memory and dask graphs.

The graph below shows the percentage of data remaining each year after per-pixel masking.

In [ ]:
# Per-pixel tide masking
pixel_tide_masked = mask_pixels_by_tide(filtered, tide_data_location, tide_centre)

In [ ]:
# Plot the percentage of pixels remaining after high-res tide masking
filter_null_byyear = filtered.mndwi.isnull().groupby("time.year").sum(dim=["time", "x", "y"])
masked_null_byyear = pixel_tide_masked.mndwi.isnull().groupby("time.year").sum(dim=["time", "x", "y"])
diff = (filter_null_byyear / masked_null_byyear) * 100
diff.plot.line(x="year", ylim=[0, 100], figsize=(12, 4), yticks=range(0, 101, 20))

## Create annual summary datasets

Here we take the daily data and combine it into one- and three-year composites, before
gapfilling the one-year composite with the three-year composite where there are
not enough observations.

In [ ]:
# Create a yearly dataset, loaded into memory. This takes a long time!
combined_ds = generate_yearly_composites(pixel_tide_masked, start_year, end_year, include_nir=True, debug=True)

# Load the combined dataset instead. Make sure you comment out the filtered
# load step. This will take longer, but use less memory.
# combined_ds = combined_ds.compute()
combined_ds

In [ ]:
for year in [2012, 2015, 2018]:
    for band in ["green", "nir", "swir", "mndwi"]:
        combined_ds.sel(year=year)[band].odc.write_cog(f"head_{year}_{band}.tif", overwrite=True)

In [ ]:
combined_ds.mndwi.plot.imshow(col="year", col_wrap=4, cmap="RdBu_r", vmin=-1, vmax=1)

In [ ]:
include_nir_land = True
nir_threshold = 0.2

thresholded_ds = combined_ds.copy(deep=True)
# Set any pixels with only one observation to NaN, as these are
# extremely vulnerable to noise
masked_ds = thresholded_ds.where(combined_ds["count"] > 1)

# Apply water index threshold and re-apply nodata values
nodata = masked_ds[water_index].isnull()
ndwi_land = masked_ds[water_index] < index_threshold

# Include a NIR threshold, from DE Pacific's example
if include_nir_land:
    nir_land = combined_ds.nir < nir_threshold
    thresholded_ds = ndwi_land & nir_land
else:
    thresholded_ds = ndwi_land

# Mask out the nodata
thresholded_ds = thresholded_ds.where(~nodata)

In [ ]:
nir_land = combined_ds.nir > nir_threshold
nir_land.plot.imshow(col="year", col_wrap=4, cmap="Greys_r", vmin=0, vmax=1)

In [ ]:
ndwi_land = masked_ds[water_index] > index_threshold
ndwi_land.plot.imshow(col="year", col_wrap=4, cmap="Greys_r", vmin=0, vmax=1)

In [ ]:
combined_ds.sel(year=2018).green.plot.hist(bins=100, range=[0, 0.1])

In [ ]:
thresholded_ds = ndwi_land & nir_land
thresholded_ds.plot.imshow(col="year", col_wrap=4, cmap="Greys_r", vmin=0, vmax=1)

In [ ]:
nir_land.sel(year=2015).plot.imshow(cmap="Reds", vmin=0, vmax=1, size=6)

In [ ]:
ndwi_land.sel(year=2015).plot.imshow(cmap="Greys_r", vmin=0, vmax=1, size=6)

In [ ]:
both_land = nir_land & ndwi_land
both_land.sel(year=2019).plot.imshow(cmap="Blues", vmin=0, vmax=2, size=6)

## Load vector data and pre-process raster data

In [ ]:
# Coastal mask modifications with "add" and "remove" areas
modifications_gdf = gpd.read_file(
    config["Input files"]["modifications_path"], bbox=bbox
).to_crs(str(combined_ds.odc.crs))

# Mask dataset to focus on coastal zone only
(
    masked_ds,
    certainty_masks,
    all_time_20,
    all_time_80,
    river_mask,
    ocean_da,
    thresholded_ds,
    temporal_mask,
    annual_mask,
    coastal_mask,
    ocean_mask,
) = contours_preprocess(
    combined_ds=combined_ds,
    water_index=water_index,
    index_threshold=index_threshold,
    buffer_pixels=50,
    mask_with_esa_wc=True,
    modifications_gdf=modifications_gdf,
    debug=True,
    # include_nir_land=True,
    # nir_threshold=0.2
)

# Plot a single timestep
masked_ds.sel(year=2017).plot.imshow(size=8, cmap="RdBu")

In [ ]:
# Extract shorelines
contours = subpixel_contours(
    da=masked_ds,
    z_values=index_threshold,
    min_vertices=10,
    dim="year",
).set_index("year")

if len(contours) == 0:
    raise ValueError("No contours found. Try a lower index threshold.")

# Add quality measures
contours_with_certainty = contour_certainty(contours, certainty_masks)

# Change the year index to a normal geopandas field
plot_contours = contours_with_certainty.reset_index()

# Plot shorelines
plot_contours.explore(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    cmap="magma",
    column="year"
)

In [ ]:
raise Exception("Pop")

## Compute statistics

###  Create stats points on baseline shorline

In [ ]:
# Extract statistics modelling points along baseline shoreline
try:
    points = points_on_line(contours_with_certainty, baseline_year, distance=30)
except KeyError:
    print("Failed to make points")
    points_gdf = None

### Measure annual coastline movements

In [ ]:
if points is not None and len(points) > 0:
    # Calculate annual movements for every shoreline
    # compared to the baseline year
    points = annual_movements(
        points,
        contours_with_certainty,
        combined_ds,
        str(baseline_year),
        water_index,
        max_valid_dist=1200,
    )
    
    # Reindex to add any missing annual columns to the dataset
    points = points.reindex(
        columns=[
            "geometry",
            *[f"dist_{i}" for i in range(start_year, end_year + 1)],
            "angle_mean",
            "angle_std",
        ]
    )
else:
    print("Something went wrong! Check the points.")

### Calculate regressions

In [ ]:
if points is not None and len(points) > 0:
    # Apply regression function to each row in dataset
    points = calculate_regressions(points)

    # Add count and span of valid obs, Shoreline Change Envelope (SCE),
    # Net Shoreline Movement (NSM) and Max/Min years
    stats_list = ["valid_obs", "valid_span", "sce", "nsm", "max_year", "min_year"]
    points[stats_list] = points.apply(
        lambda x: all_time_stats(x, initial_year=start_year), axis=1
    )

    # And add certainty
    points_with_certainty = points_certainty(points, baseline_year=baseline_year)

## Export files

In [ ]:
clipped_points_gdf = points_with_certainty.clip(geometry.to_crs(points_with_certainty.crs))
clipped_contours_gdf = contours_with_certainty.clip(geometry.to_crs(contours_with_certainty.crs))

out_files = export_results(clipped_points_gdf, clipped_contours_gdf, version, str(output_dir), study_area)

for out_file in out_files:
    print(f"Exported {out_file}")

### Close Dask client

In [ ]:
dask_client.close()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984